In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# pour afficher des images 
from IPython.display import Image

# pour definir le chemin vers le work space
import os

# Pour le traitements de données après les avoir récoltées 
import string 
import re

# chemin vers mon work space
os.chdir("C:\\Users\\baptc\\Documents\\GM\\optimisation disc\\projet")

# pour afficher les données sans faire de print
get_ipython().run_line_magic('matplotlib', 'inline')

Nous n'avons pas trouvé d'instance de test sur internet. Nous avons donc commencé à fabriquer nous-même nos jeux de données.
Pour cela, nous avons choisi de créer notre propre GPS. Nous voulions faire un algorithme qui permette de minimiser un paramètre sous la contrainte des autres paramètres. Les paramètres que nous avons retenus sont les suivants :
    
La distance, le temps, le coût lié au carburant et le coût lié aux péages.

On peut trouver de nombreux fichiers sur internet dans lesquels on peut avoir toutes les distances entre toutes les communes de France.
Dans un premier temps, nous avons fait un programme de scraping qui avait récupéré l'ensemble de ces distances. Puis, nous avons fait un graphe entre une quinzaine de villes en allant chercher sur Mappy les ressources.
Mais en faisant cela, nous relions toutes les villes les unes entre les autres et donc nous avions que des chemins de longueur 1 entre toutes les villes. Le problème de minimiser sous contrainte dans ce cas revenait à prendre un cas particulier et donc à toujours prendre le chemin de longueur 1. Donc pas intéressant comme instance de test.

Nous avons donc compris que l'on ne pouvait pas faire un jeu de données à l'aveugle en prenant des villes et en les reliant aléatoirement.

Dans la suite nous avons créé à la main, un graphe des principales villes françaises. Nous avons essayé de ne pas relier au hasard les communes françaises. Nous avons essayé de ne garder que les arcs qui vérifiaient pour (a,b,c)$\in$S si a$\rightarrow$b et b$\rightarrow$c alors nous n'avons pas mis l'arc a$\rightarrow$c. Pour éviter les "cours circuits" et les chemins de longueurs. Par exemple, un cas extrême que nous avons aussi respecté localement ce serait d'avoir un arc Lille, Marseille. Cela est sans intérêt. Nous relions des villes aux villes voisines les plus proches.

Enfin pour récupérer rapidement les variables de notre modèle. Nous avons fait un BOT qui permet d'aller faire du scraping sur Mappy. Nous vous présenterons plus tard ces algorithmes.


Ci-dessous, nous avons mis dans un fichier txt tous les arcs que nous avons sélectionnés pour faire notre réseau routier.

Nous avons choisi, des villes de manière à avoir plusieurs chemins entre des villes en passant ou non pas des péages pour que les contraintes soient actives. Sur Mappy, nous avons récupéré les valeurs qui correspondent au plus court chemin en temps entre deux villes. Dès que possible le GPS prend l'autoroute donc le péage car c'est souvent par-là que passe le plus court chemin en temps. C'est pourquoi nous avons aussi mis des petites villes pour relier des grandes villes et donc avoir des chemins alternatifs plus long en distance mais avec moins de contraintes de péage. Nous avons fait aussi attention de ne pas cours circuiter des villes.
Le choix des villes a été une partie importante pour la réalisation de ce GPS.


In [2]:
data = pd.read_table("Reseaux_routier_3.txt", sep=";")
data

,Ville_depart,Ville_arrivee
0,GENEVE,DIJON
1,NANTES,BREST
2,SAINT-BREVIN,NANTES
3,TARBES,TOULOUSE
4,TARBES,PAU
...,...,...
93,BESANCON,GENEVE
94,BORDEAUX,LIMOGES
95,LILLE,AMIENS
96,AMIENS,LAON


# scraping

Pour faire ce bot, nous avons utiliser le package Selenium, qui utilise un driver. Nous avons du installer sur notre ordinateur  le driver suivant :

In [3]:
Image("C:\\Users\\baptc\\Documents\\GM\\optimisation disc\\projet\\driver.JPG")

On peut l'installer simplement mais sans ce driver le code suivant ne va pas compiler.

In [4]:
# Package pour le BOT

import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Dans le Bot nous utiliserons le package time qui permet de marquer des pauses
# dans l'execution du programme car si on envoie trop de requete alors on se fait
# bloquer notre adresse IP et on ne peut plus accèder au site.
# pour cela nous avons marqué des remps d'arrêt aléatoire entre nos requetes

import time
from random import *

La fonction si dessous prend en argument, le dataframe dans que nous allons retourner, la liste des valeurs que nous allons ajouter au dataframe et il nous faut aussi le nom des colonnes du dataframe dans la façon dont nous avons codé cette fonction.

In [5]:
def ajout_ville(data, liste, nom_colonne):
    if not (
        (liste[0] in data.iloc[:, 0])
        and (liste[1] in data.iloc[:, 1])
        or (liste[1] in data.iloc[:, 0])
        and (liste[0] in data.iloc[:, 1])
    ):
        df_new_line = pd.DataFrame([liste], columns=nom_colonne)
        data = pd.concat([data, df_new_line], ignore_index=True)
    return data

Plutot que de traiter les données une fois celle ci recoltées. Nous avons choisi de les mettre aux formats que nous voulions. Dans le BOT nous avons récupéré par exemple des valeurs sous forme de String et ils fallaient envisager toutes les possibilités et les caster en int.

Une fonction pour mettre le temps en minute qui prend un string et retourne un int

In [6]:
def dureeToMin(heure):
    reg = re.compile("(.*) h (.*)").search(heure)
    if reg != None:
        return int(reg[1]) * 60 + int(reg[2])
    reg = re.compile("(.*) min").search(heure)
    if reg != None:
        return int(reg[1])

Une fonction qui prend le carburant en string et retourne un int

In [7]:
def carbuToInt(carbu):
    reg = re.compile("(.*),(.*) €").search(carbu)
    if reg != None:
        return int(reg[1]) + int(reg[2]) * 0.01
    reg = re.compile("(.*) €").search(carbu)
    if reg != None:
        return int(reg[1])

Une fonction qui prend une distance en string et retourne un int

In [8]:
def distanceToInt(distance):
    reg = re.compile("(.*),(.*) km").search(distance)
    if reg != None:
        return int(reg[1]) + int(reg[2]) * 0.01
    reg = re.compile("(.*) km").search(distance)
    if reg != None:
        return int(reg[1])

Une fonction qui prend le coût des péages en string et retourne un int

In [9]:
def peageToInt(peage):
    reg = re.compile("(.*),(.*) €").search(peage)
    if reg != None:
        return int(reg[1]) + int(reg[2]) * 0.01
    reg = re.compile("(.*) €").search(peage)
    if reg != None:
        return int(reg[1])

Voici le code du BOT

Quand on fait un BOT, les sites ont souvent des technos anti-BOT et donc si on fait trop de requete, qu'on reste connecté trop longtemps où que l'on fait trop de connexion ou deconnexion alors le progamme échoue avec une erreur de type "impossible d'établir une connexion vers le site cible"

Pour cela, nous avons fait en sorte que si le programme plante alors il se relance lui meme à l'endroit où il a planté.

In [ ]:
"""
Pour parcourir toutes les villes nous avons fait une boucle for. 
Cependant, la condition dans le for ne peut être dynamique.
C'est pourquoi nous avons défini cpt1 en globable qui va permet
au relancement (on relance la boucle for donc plus de problème de condition
dynamique) de l'algorithme de reprendre là où l'on était rendu

"""
global cpt1
cpt1 = 0

# création et initialisation du dataframe de retour

nom_colonne = ["depart", "arrive", "distance", "carbu", "peage", "duree"]
villes = pd.DataFrame(columns=nom_colonne)

# on va faire tourner l'algo tant que l'on a pas scrapé toutes les villes

while cpt1 != data.shape[0]:
    # Si on a une erreur de deconnexion alors on a une exception et on relance
    # la boucle while a partir de cpt1
    try:
        liste = []
        PATH = "C:\Program Files (x86)\chromedriver.exe"
        cpt = cpt1

        # on créer une instance driver pour interagir avec google chrome

        driver = webdriver.Chrome(PATH)
        driver.get("https://fr.mappy.com/itineraire")

        # on clique sur le bouton pour accepter les cookies "premiere securitée"
        # contre les BOTs

        try:
            # Si la connexion n'est pas bonne on laisse 10 secondes pour
            # que la page s'actualise

            element = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located(
                    (By.ID, "didomi-notice-agree-button")
                )  # on peut aussi mettre CLASS_NAME
            )
            element.click()
        except:
            driver.quit()

        for i in range(cpt, data.shape[0]):
            # on entre dans la barre de recherche le nom de la ville de départ
            try:
                search = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located(
                        (
                            By.XPATH,
                            "//input[@class='src-domain-header-SearchInput__SearchInput_field--2IEPV src-domain-header-SearchInput__SearchInput_minimized--v18iV src-domain-itinerary-ItineraryHeaderForm__SearchInput_default--vNJTa src-domain-itinerary-ItineraryHeaderForm__SearchInput_unfocus--1Dph7 src-domain-itinerary-ItineraryHeaderForm__SearchInput_departure--30n_-']",
                        )
                    )  # on peut aussi mettre CLASS_NAME
                )
                search.clear()
                search.send_keys(data["Ville_depart"][i])
                search.send_keys(Keys.RETURN)

            except:
                print("erreur ville depart " + data["Ville_depart"][i])

            liste.append(data["Ville_depart"][i])

            # on entre dans la barre de recherche le nom de la ville de d'arrivée
            try:
                search = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located(
                        (
                            By.XPATH,
                            "//input[@class='src-domain-header-SearchInput__SearchInput_field--2IEPV src-domain-header-SearchInput__SearchInput_minimized--v18iV src-domain-itinerary-ItineraryHeaderForm__SearchInput_default--vNJTa src-domain-itinerary-ItineraryHeaderForm__SearchInput_unfocus--1Dph7 src-domain-itinerary-ItineraryHeaderForm__SearchInput_arrival--3T2pX']",
                        )
                    )  # on peut aussi mettre CLASS_NAME
                )
                search.clear()
                search.send_keys(data["Ville_arrivee"][i])
                search.send_keys(Keys.RETURN)

            except:
                print("erreur ville d'arrivée " + data["Ville_arrivee"][i])

            liste.append(data["Ville_arrivee"][i])

            # on clique sur un bouton pour avoir le détail du trajet

            try:
                element = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located(
                        (
                            By.CLASS_NAME,
                            "src-domain-itinerary-GoToRoadbookButton__GoToRoadbookButton--3Mn6c",
                        )
                    )  # on peut aussi mettre CLASS_NAME
                )
                time.sleep(1)
                element.click()
            except:
                driver.quit()

            time.sleep(4)
            # on recupere la duree, la distance, le carburant, peage

            distance = driver.find_element_by_xpath(
                "//div[@class='src-domain-itinerary-roadbook-RoadbookSummary__SummaryItem--1CJFe length']"
            )
            distance = distanceToInt(
                distance.find_element_by_class_name(
                    "src-domain-itinerary-roadbook-RoadbookSummary__SummaryItemValue--1W6Ec"
                ).text
            )

            liste.append(distance)

            carbu = driver.find_element_by_xpath(
                "//div[@class='src-domain-itinerary-roadbook-RoadbookSummary__SummaryItem--1CJFe fuel_price']"
            )
            carbu = carbuToInt(
                carbu.find_element_by_class_name(
                    "src-domain-itinerary-roadbook-RoadbookSummary__SummaryItemValue--1W6Ec"
                ).text
            )

            liste.append(carbu)

            try:
                peage = driver.find_element_by_xpath(
                    "//div[@class='src-domain-itinerary-roadbook-RoadbookSummary__SummaryItem--1CJFe toll_price']"
                )
                peage = peageToInt(
                    peage.find_element_by_class_name(
                        "src-domain-itinerary-roadbook-RoadbookSummary__SummaryItemValue--1W6Ec"
                    ).text
                )
                liste.append(peage)
            except:
                liste.append(0)

            duree = driver.find_element_by_xpath(
                "//div[@class='src-domain-itinerary-roadbook-RoadbookSummary__SummaryItem--1CJFe duration']"
            )
            duree = dureeToMin(
                duree.find_element_by_class_name(
                    "src-domain-itinerary-roadbook-RoadbookSummary__SummaryItemValue--1W6Ec"
                ).text
            )

            liste.append(duree)

            cpt += 1

            # on ajoute la villes

            villes = ajout_ville(villes, liste, nom_colonne)

            # on retourne en arrière en cliquant sur le bouton de retour
            driver.back()
            driver.back()
            liste.clear()
        cpt1 = cpt
    except:
        cpt1 = cpt
driver.quit()

In [12]:
villes

,depart,arrive,distance,carbu,peage,duree
0,GENEVE,DIJON,264,21.09,26.7,160
1,NANTES,BREST,297,23.83,0.0,208
2,SAINT-BREVIN,NANTES,69,5.79,0.0,74
3,TARBES,TOULOUSE,158,12.78,8.8,113
4,TARBES,PAU,45.09,3.86,2.8,46
...,...,...,...,...,...,...
93,BESANCON,GENEVE,177,14.50,0.0,163
94,BORDEAUX,LIMOGES,222,17.98,0.0,174
95,LILLE,AMIENS,140,11.38,9.3,118
96,AMIENS,LAON,130,10.47,8.5,86


# exporter les données

In [13]:
villes.to_csv("Reseaux_routier_VF", header=True, encoding="utf-8")